In [ ]:
import boto3
import pandas as pd
from dotenv import load_dotenv
import os
from io import BytesIO


In [ ]:
load_dotenv("/mnt/c/Users/Daniel Sánchez/Desktop/ZRIVE DS/zrive-ds/src/module_2/credentials.env")

aws_access_key = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_region = os.getenv('AWS_DEFAULT_REGION')

In [ ]:
# Initialize S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_DEFAULT_REGION')
)

# Define the bucket and prefix
bucket_name = 'zrive-ds-data'
prefix = 'groceries/sampled-datasets/'

# List objects in the S3 bucket
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# Print the list of files
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No files found in the bucket.")


In [ ]:
# Define the file you want to download (e.g., orders.parquet)

orders = 'groceries/sampled-datasets/orders.parquet'
regulars ='groceries/sampled-datasets/regulars.parquet'
inventory = 'groceries/sampled-datasets/inventory.parquet'
abandoned_carts = 'groceries/sampled-datasets/abandoned_carts.parquet'
users= 'groceries/sampled-datasets/users.parquet'

# Download the Parquet file from S3

response_orders = s3.get_object(Bucket=bucket_name, Key=orders)
response_regulars = s3.get_object(Bucket=bucket_name, Key=regulars)
response_inventory = s3.get_object(Bucket=bucket_name, Key=inventory)
response_abandoned_carts = s3.get_object(Bucket=bucket_name, Key=abandoned_carts)
response_users = s3.get_object(Bucket=bucket_name, Key=users)

# Load the Parquet file into a Pandas DataFrame
df_orders = pd.read_parquet(BytesIO(response_orders['Body'].read()))
df_regulars = pd.read_parquet(BytesIO(response_regulars['Body'].read()))
df_inventory = pd.read_parquet(BytesIO(response_inventory['Body'].read()))
df_abandoned_carts = pd.read_parquet(BytesIO(response_abandoned_carts['Body'].read()))
df_users = pd.read_parquet(BytesIO(response_users['Body'].read()))

